# Import Files and Libraries

In [8]:
from time import time
import warnings
warnings.filterwarnings('ignore')

In [11]:
import os 
import sys

import pandas as pd
import numpy as np
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Input
from tensorflow.keras.optimizers import  SGD

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import RepeatedKFold, KFold
from keras.models import Sequential
from keras.layers import Dense, Input
from tensorflow.keras.initializers import  GlorotUniform, Constant, Zeros, GlorotNormal
from scikeras.wrappers import KerasClassifier, KerasRegressor
from tensorflow.keras.regularizers import l2
from sklearn.metrics import make_scorer
from tensorflow.keras.callbacks import EarlyStopping


from sklearn.datasets import make_classification

dir_path = os.getcwd().split(os.path.sep)
root_index = dir_path.index('Machine_Learning_project')
root_path = os.path.sep.join(dir_path[:root_index + 1])
sys.path.append(root_path + '/code/')
sys.path.append(root_path + '/code/data_loaders/')
sys.path.append(root_path + '/code/utils_keras')






In [12]:
# Class for the Monks datasets
from data import *
from Trainer import *

# Class for the Cup dataset
from data_cup import *
from Trainer_Cup import *


# Neural Networks with Stochastic Gradient Descent
 In this notebook, we have implemented a Neural Network with ***Stochastic Gradient Descent (SGD)*** with mini-batch optimization using the Keras library.
 
 SGD updates network parameters based on gradients computed from small subsets of randomly selected training data, striking a balance between computational efficiency and stochastic learning. The mini-batch optimization approach helps in reducing the variance of the parameter updates, leading to more stable convergence. 
 
Specifically, two separate problems were tackled:
 - 1) classification task, involving the MONK datasets, divided into three parts;
 - 2) regression task, on the other hand, involved the CUP dataset, with a final blind test. Our goal was to thoroughly design, implement, and evaluate neural network architectures against the assigned tasks.


To evaluate the performance of our neural networks, we used several metrics suitable for each task. For the MONK problems, we used **Mean Square Error (MSE)** to calculate the loss and accuracy as the metric to measure classification performance. 
For the CUP dataset, we also used Mean Square Error (MSE) for calculating the loss, while **Mean Euclidean Error (MEE)** was employed as the metric to evaluate the effectiveness of the regression model.

# Monk 1

#### Import Monk 1 datasets

In [26]:
m1_train = MonksDataset('monk1_train')
m1_test= MonksDataset('monk1_test')

#Splitting the data into train/dev, and test sets
X_dev, y_dev, X_test_m1, y_test_m1 = get_monks_data(m1_train, m1_test)



#### Encoding


In [ ]:
# Encoding with the O.H.E. method using pandas get_dummies()
X_monk_train_ohe_pd_cat = pd.get_dummies(X_dev, columns=['a1', 'a2', 'a3', 'a4', 'a5', 'a6'])
X_monk_test_ohe_pd_cat = pd.get_dummies(X_test_m1, columns=['a1', 'a2', 'a3', 'a4', 'a5', 'a6'])

#### Model define

In [ ]:
# Define the model architecture using only lr and momentum
def model_creation( learning_rate, momentum ):
  model = Sequential()
  model.add(Dense(4, activation='tanh', input_shape=(17,)))
  model.add(Dense(1, activation='sigmoid'))
  #tf.random.set_seed(54)
  model.compile(
        optimizer= SGD(learning_rate=learning_rate, momentum=momentum),
        loss='mean_squared_error',
        metrics=['accuracy']
        )

  return model


#### Grid search

In [ ]:
# Hyperparameters definition for the grid search
params = {
    'batch_size': [ 8, 16, 32],
    'model__learning_rate': [ 0.3, 0.5, 0.7, 0.8 ],
    'model__momentum': [0.3, 0.4, 0.5, 0.7, 0.8]
}


In [ ]:
%time
# Wrap model creation in KerasClassifier
model = KerasClassifier(model=model_creation, epochs=100, metrics='accuracy_score', random_state=42, verbose=0)
seed = 42
tf.random.set_seed(seed)

# Grid search
grid = GridSearchCV(estimator=model, param_grid=params, cv=5, error_score='raise', n_jobs=-1, verbose=0)
grid_result = grid.fit(X_monk_train_ohe_pd_cat, y_dev, verbose=0)

# Get best hyperparameters
print("Best hyperparameters:", grid_result.best_params_)

#### Training

In [ ]:
# Model creation and training
model_monk1 =model_creation(0.8,0.8)
model_trained_m1 = Trainer(model_monk1, X_monk_train_ohe_pd_cat, y_dev, X_monk_test_ohe_pd_cat, y_test )
model_trained_m1.train(epochs=300, batch_size=8) # Trainer functions
model_trained_m1.plot_history('Monk1')  # Plotting of the training history

#### Evaluation

In [ ]:
model_trained_m1.evaluate(X_monk_train_ohe_pd_cat, y_dev)


In [ ]:
model_trained_m1.evaluate(X_monk_test_ohe_pd_cat, y_test_m1)

# Monk2

In [ ]:
m2_train = MonksDataset('monk2_train')
m2_test= MonksDataset('monk2_test')

In [ ]:
X_dev_m2, y_dev_m2, X_test_m2, y_test_m2 = get_monks_data(m2_train, m2_test)


#### Encoding

In [ ]:
X_monk2_dev_ohe_pd_cat = pd.get_dummies(X_dev_m2, columns=['a1', 'a2', 'a3', 'a4', 'a5', 'a6'])
X_monk2_test_ohe_pd_cat = pd.get_dummies(X_test_m2, columns=['a1', 'a2', 'a3', 'a4', 'a5', 'a6'])

#### Grid Search

In [ ]:
params = {
    'batch_size': [8, 16, 32, 64, len(X_dev_m2)],
    'model__learning_rate': [0.1 ,0.3, 0.5, 0.7 ],
    'model__momentum': [  0.5, 0.7, 0.8, 0.9],
}

In [ ]:
%time
# Wrap model creation in KerasClassifier
model = KerasClassifier(model=model_creation, epochs=50, verbose=0)
seed = 42
tf.random.set_seed(seed)

# Grid search
grid = GridSearchCV(estimator=model, param_grid=params, cv=5, error_score='raise' )
grid_result = grid.fit(X_monk2_dev_ohe_pd_cat, y_dev_m2)

# Get best hyperparameters
print("Best hyperparameters:", grid_result.best_params_)

#### Training

In [ ]:
model_monk2 =model_creation(0.5, 0.7)
model_trained_m2 = Trainer(model_monk2, X_monk2_dev_ohe_pd_cat, y_dev_m2, X_monk2_test_ohe_pd_cat, y_test_m2 )
model_trained_m2.train(epochs=300, batch_size=8)
model_trained_m2.plot_history('Monk2')

#### Evaluation

In [ ]:
model_trained_m2.evaluate(X_monk2_dev_ohe_pd_cat, y_dev_m2)

In [ ]:
model_trained_m2.evaluate(X_monk2_test_ohe_pd_cat, y_test_m2)

# Monk 3

### Import the dataset

In [ ]:
m3_train = MonksDataset('monk3_train')
m3_test= MonksDataset('monk3_test')

In [ ]:
X_dev_m3, y_dev_m3, X_test_m3, y_test_m3 = get_monks_data(m3_train, m3_test)

#### Encoding

In [ ]:
X_monk3_dev_ohe_pd_cat = pd.get_dummies(X_dev_m3, columns=['a1', 'a2', 'a3', 'a4', 'a5', 'a6'])
X_monk3_test_ohe_pd_cat = pd.get_dummies(X_test_m3, columns=['a1', 'a2', 'a3', 'a4', 'a5', 'a6'])


### Grid search without regularization

In [ ]:
params = {
    'batch_size': [8, 16],
    'model__learning_rate': [ 0.01, 0.1 ,0.3, 0.5],
    'model__momentum': [  0.7, 0.9]
}

In [ ]:
%time
# Wrap model creation in KerasClassifier
model = KerasClassifier(model=model_creation, epochs=300, verbose=0)
seed = 42
tf.random.set_seed(seed)

# Grid search
grid = GridSearchCV(estimator=model, param_grid=params, cv=5, error_score='raise', n_jobs=-1)
grid_result = grid.fit(X_monk3_dev_ohe_pd_cat, y_dev_m3)

# Get best hyperparameters
print("Best hyperparameters:", grid_result.best_params_)

#### Training

In [ ]:
model_monk3 =model_creation(0.01, 0.7)
model_trained_m3 = Trainer(model_monk3, X_monk3_dev_ohe_pd_cat, y_dev_m3, X_monk3_test_ohe_pd_cat, y_test_m3 )
model_trained_m3.train(epochs=300, batch_size=8)
model_trained_m3.plot_history('Monk3')

#### Evaluation

In [ ]:
model_trained_m3.evaluate(X_monk3_dev_ohe_pd_cat, y_dev_m3)

In [ ]:
model_trained_m3.evaluate(X_monk3_test_ohe_pd_cat, y_test_m3)

# Monk 3 with regularization

### Model define

In [ ]:
# Model creation with the addition of searching for the best value of regularization
def model_creation_regularizer( learning_rate, momentum, regularizers ):
  model = Sequential()
  model.add(Dense(4, activation='tanh', input_shape=(17,),kernel_regularizer=l2(regularizers)))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(
        optimizer= SGD(learning_rate=learning_rate, momentum=momentum),
        loss='mean_squared_error',
        metrics=['accuracy']
        )

  return model

#### Grid Search with regularization

In [ ]:
# Definition of hyerparameters to search
params = {
    'batch_size': [ 32, 64, 128],
    'model__learning_rate': [ 0.3, 0.4, 0.5],
    'model__momentum': [ 0.7,0.8, 0.9],
    'model__regularizers': [ 0.01, 0.001, 0.0001]
}


In [ ]:
%time
# Definition of the model
model = KerasClassifier(model=model_creation, epochs=600, verbose=0)
seed = 42
tf.random.set_seed(seed)

# Grid search
grid = GridSearchCV(estimator=model, param_grid=params, cv=5, error_score='raise', n_jobs=-1, verbose=0)
grid_result = grid.fit(X_monk3_dev_ohe_pd_cat, y_dev_m3)

# Get best hyperparameters
print("Best hyperparameters:", grid_result.best_params_)

### Training

In [ ]:
model_monk3_reg =model_creation_regularizer(0.1, 0.3, 0.01)
model_trained_m3_reg = Trainer(model_monk3_reg, X_monk3_dev_ohe_pd_cat, y_dev_m3, X_monk3_test_ohe_pd_cat, y_test_m3 )
model_trained_m3_reg.train(epochs=600, batch_size=32)
model_trained_m3_reg.plot_history('Monks 3 with reg')

#### Evaluation


In [ ]:
model_trained_m3_reg.evaluate(X_monk3_dev_ohe_pd_cat, y_dev_m3)

In [ ]:
model_trained_m3_reg.evaluate(X_monk3_test_ohe_pd_cat, y_test_m3)

# Cup

In [14]:
# Create an instance of the dataset for Cup training and Cup test
cup = CupDataset('Cup_tr')
blind = CupDataset('Cup_ts')

blind = blind.data
df =cup.data





#### Train/Val - Test Split
 The dataset are splitted into 3 part: Train, Val, Test. The Dev set(90%), include Train(90%) and Val(10%) for model selection, and Test set(10%) is used for final evaluation for model assessment.

In [15]:
# Split data into train/validation and test sets
cup.split_data(test_size=0.1, random_state=0)

# X_dev and y_dev represent the features and labels of the development set (train/validation combined), X_final_test and y_final_test represent the features and labels of the final test set
X_dev,  X_final_test, y_dev, y_final_test = cup.get_splits()

# Further split the development set (X_dev, y_dev) into training and internal test sets
X_train, X_internal_test, y_train, y_internal_test = train_test_split(X_dev, y_dev, test_size=0.111, random_state=0)

# Extract the features from the 'blind' dataset 
X_blind = blind[['a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'a10']]

# Preprocesing
To preprocess the data, a polynomial transformation (***PolynomialFeatures***) was applied using the polynomial degree fixed. Next, the ***arctanh*** (hyperbolic arcotangent) function was applied to scale the data.

In [16]:
poly = PolynomialFeatures(degree=2)
X_train_poly = np.arctanh(poly.fit_transform(X_train)[:,1:])
X_internal_test_poly = np.arctanh(poly.transform(X_internal_test)[:,1:])
X_final_test_poly = np.arctanh(poly.transform(X_final_test)[:,1:])
X_dev_poly = np.arctanh(poly.transform(X_dev)[:,1:])
X_blind_poly = np.arctanh(poly.transform(X_blind)[:,1:])


# Model Selection and Hyperparameters tuning
An initial broad grid search is typically performed to explore a wide array of hyperparameter values, aiming to pinpoint the most promising regions within the parameter space. Once these regions are identified, a subsequent, more detailed grid search is conducted to precisely optimize the hyperparameter settings.

 1)  A ***coarse grid search***,  was initially conducted, for the model selection using primarily to determine the optimal hyperparameters for the model architecture.
 2) A more ***fine grid search*** was performed to accurately identify the optimal learning hyperparameters.

The optimal configurations for final re-training and evaluation on the internal test set are those that achieve the lowest mean MEE during cross-validation on the validation set.


In [17]:
# Definition of function to estimate the metric MEE(Mean Euclidean Error)
def mean_euclidean_error(y_true, y_pred):
  return tf.reduce_mean(tf.sqrt(tf.reduce_sum(tf.square(y_true - y_pred), axis=-1)))


In [18]:
# Definition of the function for the model creation to use for the grid search on the archicture parameters
initializer = GlorotUniform(seed=12)
in_dim = X_train_poly.shape[1]
out_dim = y_train.shape[1]

def create_mlp_model(activation, num_hidden_layers, h_units, learning_rate, momentum, regularizers):
    # create model
    model = Sequential()
    model.add(Input(shape=(in_dim,)))

    # # Add the hidden layers.
    for i in range(1, num_hidden_layers+1):
        model.add(Dense(h_units, activation=activation,kernel_initializer=initializer, kernel_regularizer=l2(regularizers)) )

    model.add(Dense(out_dim, activation='linear'))

    # Compile model
    model.compile(
        optimizer= SGD(learning_rate=learning_rate, momentum=momentum, clipnorm=10.0),
        loss='mean_squared_error',
        metrics=[mean_euclidean_error]
        )

    return model

In [27]:
# Definition of the early stopping criteria
early_stopping = EarlyStopping(
                              patience=50,
                              monitor="mean_euclidean_error",
                              mode='min',
                              restore_best_weights=True,
                              min_delta=0.01,
                              verbose=1
                              )

In [35]:
# Definition of the hyperparameters to search for the corase grid search
params = {
    'batch_size': [64, 128],
    'model__num_hidden_layers': [2,3],
    'model__h_units': [ 60, 120, 150],
    'model__learning_rate': [0.01, 0.001, 0.0001],
    'model__momentum': [0.0, 0.3, 0.6, 0.9],
    'model__activation': ['relu', 'tanh'],
    'model__regularizers': [0.001, 0.01, 0.0001]
}

In [ ]:
tf.keras.utils.set_random_seed(42)
n_jobs_search = -1


# Definition of the model to use for the grid search, the grid search will be done on the architecture parameters using the cross validation on 5 and MEE as scorer

mlp = KerasRegressor(
                    model=create_mlp_model,
                    epochs=800,
                    callbacks=[early_stopping],
                    )

mlp_cv_m1 = GridSearchCV(estimator=mlp, param_grid=params, scoring=make_scorer(lambda x, y : mean_euclidean_error(x, y).numpy()), cv=5, verbose=0,
                        n_jobs=n_jobs_search, error_score='raise')

grid_result = mlp_cv_m1.fit(X_train_poly, y_train, verbose =0 )

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
results = sorted(zip(means, stds, params), key=lambda x: x[0], reverse=False)
for mean, stdev, param in results:
    print("%f (%f) with: %r" % (mean, stdev, param)) 

In [ ]:
# Here the results of the coarse grid search are printed and sorted
results = sorted(zip(means, stds, params), key=lambda x: x[0], reverse=False)
for mean, stdev, param in results:
    print("%f (%f) with: %r" % (mean, stdev, param)) 

### Fine-Grid search on hyperparameters

In [17]:
initializer = GlorotUniform(seed=12)
in_dim = X_train_poly.shape[1]
out_dim = y_train.shape[1]
# Definition of the model for the fine grid search
def create_mlp_model_arch( learning_rate, momentum, regularizers):
    

    model = Sequential()
    model.add(Input(shape=(in_dim,)))
    for _ in range(3):
        model.add(Dense(150, activation='tanh', kernel_initializer=initializer, kernel_regularizer=l2(regularizers)))
    
    model.add(Dense(out_dim, activation='linear'))

    model.compile(
        optimizer=SGD(learning_rate=learning_rate, momentum=momentum, clipnorm=10.0),
        loss='mean_squared_error',
        metrics=[mean_euclidean_error]
    )

    return model

In [ ]:
# Definition of hyperparameters for the fine grid search
param_dist = {
    'batch_size': (32, 64, 128),
    'model__learning_rate':(0.01, 0.008, 0.005, 0.003, 0.001),
    'model__momentum': (0.5, 0.6, 0.7),
    'model__regularizers': (0.0001, 0.0005, 0.00005, 0.0001)
}

In [ ]:
model = KerasRegressor(model=create_mlp_model_arch, verbose=0, epochs=800, callbacks=[early_stopping])

grid_fine_search = GridSearchCV(estimator=model, param_grid=param_dist, scoring=make_scorer(lambda x, y : mean_euclidean_error(x, y).numpy()), n_jobs=-1, cv=5, verbose=3)
grid_fine_search.fit( X_train_poly, y_train)

# Stampa i risultati della random search
print("Best parameters found:")
print(grid_fine_search.best_params_)
print("Best score: ", grid_fine_search.best_score_)

In [ ]:
#Get the cross-validation results for the fine grid search and sort them by the best mean test score
results = grid_fine_search.cv_results_

sorted_indices = np.argsort(results['mean_test_score'])

num_best_results = 5

print(f"Best {num_best_results} Results:")
for i in range(num_best_results):
    index = sorted_indices[i]
    print(f"Rank {i+1}:")
    print(f"  Parameters: {results['params'][index]}")
    print(f"  Mean Test Score: {results['mean_test_score'][index]:.4f}")
    print(f"  Std Test Score: {results['std_test_score'][index]:.4f}")
    print()



## Model assessment
After identifying the optimal hyperparameter configuration using the two different grid searches, we proceed with training our model on the training/validation set. This approach allows us to fully utilize all available training/validation data. 

#### Training best model

In [ ]:
# Training the best model found by the fine grid search and plotting the learning curves
model = create_mlp_model('tanh', 3, 150, 0.003, 0.7, 0.0001)
model_trained = Trainer_cup(model, X_train_poly, y_train, X_internal_test_poly, y_internal_test, use_early_stopping=True, early_stopping_patience=50) 
model_trained.train(epochs=1200, batch_size=32, verbose=0)
model_trained.plot_history('Model SGD with KerasRegressor')
# Save the model withe the best Hyperparameters found
#model_trained.save_model('model_trained_1.keras')

## Training Average to estimate the error of the model

To achieve a more precise estimate on the validation test, internal test, than on the blind test, we conduct 5 trials with identical settings and then calculate the mean.

##### Model trained on train of validation and estimate error on validation test set

In [ ]:
#Array to store predictions of development and test data for each trial
train_preds_arr = np.zeros((5, len(y_train), 3))
internal_test_preds_arr = np.zeros((5, len(y_internal_test), 3))

# Perform 5 training trials
for i in range(5):
    
    model_train = create_mlp_model('tanh', 3, 150, 0.003, 0.7, 0.0001)
    
    model_trained = Trainer_cup(model_train, X_train_poly, y_train,  use_early_stopping=True, early_stopping_patience=50)
    
    model_trained.train(epochs=1200, batch_size=32, verbose=0)
    
    train_preds_arr[i] = model_trained.predict(X_train_poly)
    
    internal_test_preds_arr[i] = model_trained.predict(X_internal_test_poly)
    
    
    test_loss, test_mee = model_trained.evaluate(X_internal_test_poly, y_internal_test)
    print(f"Trial {i+1} - Test Loss: {test_loss:.4f}, Test MEE: {test_mee:.4f}")


train_preds_mean = np.mean(train_preds_arr, axis=0)
internal_test_preds_mean = np.mean(internal_test_preds_arr, axis=0)


print("Mean Dev Predictions:")
print(train_preds_mean)
print("Mean Final Test Predictions:")
print(internal_test_preds_mean)

Train_test_preds_df = pd.DataFrame(train_preds_mean, columns=['x', 'y', 'z'])
Train_test_preds_df.to_csv('Train_avg_predictions.csv', index=False)

internal_test_preds_df = pd.DataFrame(internal_test_preds_mean, columns=['x', 'y', 'z'])
internal_test_preds_df.to_csv('Internal_test_avg_predictions.csv', index=False)


In [ ]:
print('-- TRAIN (MEAN 5 TRIALS) --')
train_preds = np.mean(train_preds_arr, axis=0)
loss_train_mean = np.mean(np.square(y_train - train_preds))
mee_train_mean = np.mean(np.sqrt(np.sum(np.square(y_train - train_preds), axis=1)))
print(f'Mean Loss (MSE): {loss_train_mean:.4f} - Mean MEE: {mee_train_mean:.4f}')


In [ ]:
print('-- INTERNAL TEST (MEAN 5 TRIALS) --')
loss_internal_test_mean = np.mean(np.square(y_internal_test - internal_test_preds_mean))

mee_internal_test_mean = np.mean(np.sqrt(np.sum(np.square(y_internal_test - internal_test_preds_mean), axis=1)))
print(f'Mean Loss (MSE): {loss_internal_test_mean:.4f} - Mean MEE: {mee_internal_test_mean:.4f}')


##### Model trained on all validation set and estimate error on final test set

In [ ]:
#Array to store predictions of development and test data for each trial
dev_preds_arr = np.zeros((5, len(y_dev), 3))
final_test_preds_arr = np.zeros((5, len(y_final_test), 3))

# Perform 5 training trials
for i in range(5):
    
    model_dev = create_mlp_model('tanh', 3, 150, 0.003, 0.7, 0.0001)
    
    
    model_trained_dev = Trainer_cup(model_dev, X_dev_poly, y_dev,  use_early_stopping=True, early_stopping_patience=50)
    
    
    model_trained_dev.train(epochs=1200, batch_size=32, verbose=0)
    
    
    dev_preds_arr[i] = model_trained_dev.predict(X_dev_poly)
    
    
    final_test_preds_arr[i] = model_trained_dev.predict(X_final_test_poly)
    
    
    test_loss, test_mee = model_trained_dev.evaluate(X_final_test_poly, y_final_test)
    print(f"Trial {i+1} - Test Loss: {test_loss:.4f}, Test MEE: {test_mee:.4f}")


dev_preds_mean = np.mean(dev_preds_arr, axis=0)
final_test_preds_mean = np.mean(final_test_preds_arr, axis=0)


print("Mean Dev Predictions:")
print(dev_preds_mean)
print("Mean Final Test Predictions:")
print(final_test_preds_mean)

dev_test_preds_df = pd.DataFrame(dev_preds_mean, columns=['x', 'y', 'z'])
dev_test_preds_df.to_csv('Dev_avg_predictions.csv', index=False)

internal_test_preds_df = pd.DataFrame(final_test_preds_mean, columns=['x', 'y', 'z'])
internal_test_preds_df.to_csv('Internal_test_avg_predictions.csv', index=False)

In [ ]:
print('-- DEV (MEAN 5 TRIALS) --')
dev_preds = np.mean(dev_preds_arr, axis=0)
loss_dev_mean = np.mean(np.square(y_dev - dev_preds))
mee_dev_mean = np.mean(np.sqrt(np.sum(np.square(y_dev - dev_preds), axis=1)))
print(f'Mean Loss (MSE): {loss_dev_mean:.4f} - Mean MEE: {mee_dev_mean:.4f}')

In [ ]:
print('-- FINAL INTERNAL TEST (MEAN 5 TRIALS) --')
loss_final_test_mean = np.mean(np.square(y_final_test - final_test_preds_mean))

mee_final_test_mean = np.mean(np.sqrt(np.sum(np.square(y_final_test - final_test_preds_mean), axis=1)))
print(f'Mean Loss (MSE): {loss_final_test_mean:.4f} - Mean MEE: {mee_final_test_mean:.4f}')


##### Model trained on all the dataset and made the predictions on the blind test set.

In [98]:
X_tot=df[['a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7','a8', 'a9', 'a10']]
y_tot=df[['t1', 't2', 't3']]
X_tot_poly = np.arctanh(poly.transform(X_tot)[:,1:])


In [ ]:
tot_preds_arr = np.zeros((5, 1000, 3))
blind_preds_arr = np.zeros((5, 900, 3))
# Perform 5 training trials
for i in range(5):
    # Train (early stopping on validation/internal test MEE)
    model =  modello_2L = create_mlp_model('tanh', 3, 150, 0.003, 0.7, 0.0001)
    model_init = Trainer_cup(model, X_tot_poly, y_tot, target='mean_euclidean_error', use_early_stopping=True, early_stopping_patience=50)
    model_init.train(epochs=1200, batch_size=32, verbose=0)

    # Total dataset predictions
    tot_preds_arr[i] = model.predict(X_tot_poly)
    # Blind test predictions
    blind_preds_arr[i] = model.predict(X_blind_poly)
    

blind_average_preds = np.mean(blind_preds_arr, axis=0)
blind_test_preds_df = pd.DataFrame(blind_average_preds, columns=['x', 'y', 'z'])
blind_test_preds_df.to_csv('Blind_test_avg_predictions.csv', index=False)